# How to use honomara_web_env

honomara_web_envは、ホノルルマラソンを走る会のホームページ作成のための統合環境です。

## 1. jupyter上でのbashの操作になれる

このノートはjupyter notebookというソフトウェア上で動作しています。  
jupyter notebookというのは、人間に読みやすい形でにプログラムを実行、記録できるサービスです。  
通常はpythonを実行するために使われることが多いですが、今回は環境構築のために、bash（ターミナル環境）を使えるようにしています。  

In [ ]:
#まずは実行してみましょう
#枠の左側の再生ボタンをクリックするか、
#枠を選択した状態で Shift + Enter または Ctrl + Enterで
#枠内のコマンド（コード）を実行できます。 
echo "現在のディレクトリ"
pwd

正しく実行できたらセル（枠）の左の\[\]内に数字が表示され、

現在のディレクトリ  
/home/notebooks  

と表示されるはずです。

次に別のコマンドを実行してみましょう

In [ ]:
# Shift + Enter 等で実行してください
ls /vagrant

/vagrant中のディレクトリのファイルやディレクトリ一覧が表示されます。  
気づいた人もいるかもしれませんが、  
このディレクトリ（仮想マシン中の/vagrantディレクトリ）はホスト中のダウンロードしてきたリポジトリと同期するように設定してあります。  

複数のセル（それぞれの枠のこと）を用意したので、好きなようにコマンドを実行してみてください  
セルを追加はメニューの`insert`から可能です。  
ショートカットでもできるので、`Help`を参照してみてください。

In [ ]:
# Shift + Enter 等で実行する。
date

## 2. 各環境をセットアップする

WEB開発に関係する可能性のある様々な環境を構築する手段（スクリプト）を用意しています。  
ラインナップは以下のとおりです。
<table>
    <thead><td>項目</td><td>説明</td><td>インストール方法</td> </thead>
    <tr><td>pytho3</td><td>python3.5</td><td>(インストール済み)</td></tr>
    <tr><td>pip3</td><td>python3パッケージ管理用</td><td>(インストール済み)</td></tr>
    <tr><td>Jupyter</td><td>jupyter</td><td>(インストール済み)</td></tr>
    <tr><td>PostgreSQL</td><td>データベース</td><td>/vagrant/scripts/setup_postgres.sh</td></tr>
    <tr><td>MySQL</td><td>データベース</td><td>/vagrant/scripts/setup_mysql.sh</td></tr>
    <tr><td>Apache2&PHP</td><td>WEBサーバーとPHP7.3</td><td>/vagrant/scripts/setup_web_server.sh</td></tr>
    <tr><td>MeCab</td><td>形態素解析機</td><td>/vagrant/scripts/setup_mecab.sh</td></tr>    
    <tr><td>slim</td><td>PHP の軽量WEBフレームワーク</td><td>/vagrant/scripts/setup_slim.sh</td></tr>
    <tr><td>Flask</td><td>Python の軽量WEBフレームワーク</td><td>/vagrant/scripts/setup_flask.sh</td></tr>
    <tr><td>既存データ</td><td>PostgreSQL上にホノマラのデータをインポート</td><td>/vagrant/scripts/import_honomara_data.sh</td></tr>
</table>

### 2.1 PostgreSQLのインストール〜〜過去のデータを見てみよう〜〜
#### 2.1.1 PostgreSQLのインストール

In [ ]:
# Shift + Enter 等で実行してください
set -eu

pushd $(dirname $0)

#install postgresql
sudo apt install  postgresql-9.5 postgresql-server-dev-9.5 -y || :

# enable password login for postgresql
sudo sed -i 's/^local\s*all\s*all\s*peer/local\tall\tall\tmd5/g' /etc/postgresql/9.5/main/pg_hba.conf || :
sudo systemctl restart postgresql 


if ! (python3 -c 'import psycopg2'); then
  if ! (type pip3 > /dev/null 2>&1); then
  ./setup_pip3.sh
  fi
  sudo pip3 install psycopg2
fi

popd

最後の行に
`/vagrant/notebooks`
と表示されたら成功です。

#### 2.1.2 過去のデータのインポート

既存のデータをデータベースに導入するためににスクリプトを用意したので実行してください

In [ ]:
# Shift + Enter 等で実行してください。
# 実行完了まで数分かかります。
/vagrant/scripts/migrate_postgres.sh

INSERT 0 1 
と大量に表示されるはずです
この処理には5分くらいかかると思います。
（セルの左側の\[\]の中が＊ではなく数字になったら終了です。）
それでは実際にデータを見てみましょう

In [ ]:
# Shift + Enter 等で実行してください
export PGPASSWORD=honomara
psql -U honomara -d honomara -c 'SELECT * FROM person LIMIT 10;'

それっぽいデータが見えたら成功です。

（増改築を繰り返したテーブルに見える...、厳しい...）

### 2.2 WEBサーバーのインストール 

今回は、古くから使われておりサークルの借りているWEBサーバーでも使われているApacheとPHPをインストールします。

※
「サーバー」という単語ですが、実は複数意味があります。  
通常は「データセンターにおいてあるマシン」等を想像するかもしれません。  
ただここでは、「ユーザー（クライアント）からリクエストを受け取って応答するソフトウェア」という意味で使っています。  

In [ ]:
# Shift + Enter 等で実行してください。
# 実行完了まで数分かかります。（セルの左側の[]の中が＊ではなく数字になったら終了です。）
set -eu
sudo apt-add-repository -y ppa:ondrej/php
sudo apt update -y
sudo apt install -y  apache2
sudo apt install -y libapache2-mod-php7.3 php7.3 php7.3-cli php7.3-common
sudo apt install -y php7.3-pgsql
sudo apt install -y php7.3-mysql
sudo chmod a+rw /var/www/html


インストールできたら（セルの左の\[\]中に数字が表示されたら）ブラウザから

[127.0.0.1:8080](http://127.0.0.1:8080)
（または[localhost:8080](http://localhost:8080)）

を開いてください。恐らく  

Apache2 Ubuntu Default Page

と書いたページが表示されるはずです。

これは  
仮想マシン中の/var/www/html/index.html  
というファイルを表示したものです。



自分で作成したページを表示したい場合は
この/var/www/htmlというディレクトリにファイルを置くと、上記のURLで見ることができます。

それでは、どうやってこのディレクトリにファイルを置くか、ですが、
先程説明した様に、仮想マシン中の/vagrantディレクトリがこのgitのディレクトリと同期していることを利用します。

以下例です。

実行してみましょう。

In [ ]:
cat > /vagrant/test.html <<EOF
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title>テストタイトル</title>
</head>
<body>
    <h1>これはテストです！！</h1>
</body>
</html>
EOF
cp  /vagrant/test.html /var/www/html/

実行したら

[127.0.0.1:8080/test.html](http://127.0.0.1:8080/test.html)

を開いてみましょう

これはテストです！！

と大きく表示されていたら成功です。

なお、ディレクトリが同期されているので、自分のパソコンのFinder等でディレクトリを見るとtest.htmlが新たに生成されている事が分かると思います。

### 2.3 MySQLの設定

既存のシステムはpostgresqlというデータベースで動いていましたが、mysqlの方が入門書等が充実しているので、新しいシステムではmysqlを使います。
そこで、まずはmysqlをインストールするために以下のを実行してください。

In [ ]:

echo "mysql-server-5.7 mysql-server/root_password password honomara" |sudo debconf-set-selections -
echo "mysql-server-5.7 mysql-server/root_password_again password honomara" |sudo debconf-set-selections -
sudo apt install  mysql-server-5.7 mysql-client-5.7 -y


if ! (pythin3 -c 'import mysql.connector'); then
  if ! (type pip3 > /dev/null 2>&1); then
  ./setup_pip3.sh
  fi
  sudo pip3 install mysql-connector-python
fi

echo "create db named honomara"
mysql  -u root --password='honomara' -e "CREATE DATABASE IF NOT EXISTS honomara CHARACTER SET 'utf8';"
echo "create user named honomara"
mysql  -u root --password='honomara' -e "CREATE USER IF NOT EXISTS honomara IDENTIFIED BY 'honomara';"
echo "add ownership to user honomara on db honomara"
mysql  -u root --password='honomara' -e "GRANT ALL PRIVILEGES ON honomara.* TO  honomara;"
echo FINISH

### 2.4MeCabの設定

さて、次はMeCabという形態素解析機の導入です。
これは自然言語処理に使われるソフトなのですが、今回は過去のデータ中の名前の整形のために使います。

先程2.1.2でデータを見たときに以下の様な結果が得られたと思います。

これ、なんだか無駄が多かったり、乱雑な気がしませんか？

というわけで、これをきれいにして、姓、名、かなを抽出するために、ここでMeCabをインストールします。


person_id |    name    |       kana       | after_name | fullname | sex | class | visible 
:-: |    :-:    |       :-:       | :-: | :-: | :-: | :-: | :-: 
        21 | 松田       | a4dea4c4a4c0     | 松田       | 松田拓朗 |   0 |     9 | t
        46 | 神保       | a4b8a4f3a4dc     | 神保       | 神保友香 |   1 |     8 | f
        48 | 高松       | a4bfa4aba4dea4c4 | 高松       | 高松誠一 |   0 |     8 | f
        49 | 阪梨       | a4b5a4aba4caa4b7 | 阪梨       |          |   0 |     6 | f
        50 | 伊藤（有） | a4a4a4c8a4a6     | 伊藤       | 伊藤有仁 |   0 |     6 | f
        51 | 達         | a4bfa4c1         | 達         |          |   1 |     8 | f
        53 | 丸山       | a4dea4eba4e4a4de | 丸山       | 丸山裕樹 |   0 |     5 | f
        54 | 金原       | a4ada4f3a4d0a4e9 | 金原       |          |   0 |    10 | f
        56 | 岡本       | a4aaa4aba4e2a4c8 | 岡本       | 岡本浩之 |   0 |     6 | f
        57 | 高安       | a4bfa4aba4e4a4b9 | 高安       |          |   0 |    10 | f

以下スクリプトを実行してください。



In [ ]:
set -eu
pushd $(dirname $0)

sudo apt install mecab libmecab-dev mecab-ipadic-utf8 swig -y
sudo pip3 install mecab-python3
echo FINISHS

### 2.5 cgiの設定 （[参考リンク](https://www.server-world.info/query?os=Ubuntu_16.04&p=httpd&f=5)）

さて次は、Python用のCGIの導入です。

雑な説明で申し訳ないですが、webサイトを動かすためにpythonを使えるようになります。

In [ ]:
sudo a2enmod cgi
sudo systemctl restart apache2
[ -d /usr/lib/cgi-bin ] && sudo rmdir /usr/lib/cgi-bin
[ -d /vagrant/app ] || mkdir -p /vagrant/app
[ -e /usr/lib/cgi-bin ] || sudo ln -s /vagrant/app /usr/lib/cgi-bin
sudo chmod 755 /usr/lib/cgi-bin
cat > /usr/lib/cgi-bin/test_script <<'EOF'
#!/usr/bin/env python3
print("Content-type: text/html\n\n")
print("Hello CGI\n")
EOF
chmod 705 /usr/lib/cgi-bin/test_script
echo FINISH

実行が終了したら、以下リンク

http://127.0.0.1:8080/cgi-bin/test_script

を開いて、Hello CGI と表示されていたら成功です。

### 2.6Flaskのための各種設定

最後に、今回開発中のアプリを動かすために必要なフレームワークやら設定やらをします。(以下全て実行してください)

In [ ]:
# .htaccessを利用できるように変更
sudo sed -i -E 's/(AllowOverride\s+)None/\1All/g' /etc/apache2/apache2.conf
sudo sed -i -E 's/(AllowOverride\s+)None/\1All/g' /etc/apache2/conf-enabled/serve-cgi-bin.conf
sudo a2enmod rewrite
sudo systemctl restart apache2

cat > /var/www/html/.htaccess <<EOF
RewriteEngine On
RewriteRule ^cgi-bin\$ /cgi-bin/index.cgi [L,QSA]
EOF
chmod 604 /var/www/html/.htaccess

In [ ]:
sudo pip3 install flask flask-sqlalchemy flask-wtf flask-bootstrap flask-login flask-bcrypt

In [ ]:
sudo apt install -y language-pack-ja

In [ ]:
git clone https://github.com/progrunner17/honomara_members_site /vagrant/app/honomara_members_site

## 3.データの移行

さて、最後に既存のデータを新しいデータベースに移行したら完了です。  
以下、別のノートにデータ変換スクリプトをおいてあるので、上から準に実行してみてください。  
(それぞれ __miscellaneous info__ 以外全て実行してください。)

1. [member](member.ipynb)
2. [training](training.ipynb)
3. [after](after.ipynb)
4. [participant](participant.ipynb)
5. [race_and_results](race_and_result.ipynb)


データの移行が完了したら  
[http://127.0.0.1:8080/cgi-bin](http://127.0.0.1:8080/cgi-bin)　にアクセスすればそれっぽいサイトが見れると思います。

現在、ログイン情報は admin:admin　または honomara:honomara です。  
ログインをすると、データの登録や修正が可能になります。

自分で開発を行う際は、一度 `/vagrant/app/honomara_members_site` を削除し    
https://github.com/progrunner17/honomara_members_site を自分のgithubアカウントにforkして、    
forkしたリポジトリを /vagrant/appにクローンすればよいと思います。

なお、デバッグの際は `/var/log/apache2` 以下に複数ログがあるので見てみましょう。

In [ ]:
tail -n 10 /var/log/apache2/access.log

とはいえ、これでもかなり面倒かと思います。   
なので開発するときは

ターミナル上でappディレクトリに移動して、 `python3 app.py`を実行し、    
[http://127.0.0.1:5000](http://127.0.0.1:5000) を見ると良いでしょう。   
こうすると、デバッグ情報がWEB上で確認できるようになります。

また、SQLAlchemyのモデルを直接試したい場合は、
`/vagrant/app `ディレクトリで `python3` コマンドを実行した後

以下の様に実行すると、コンソールから直接クエリを試せます。  
Tab補完も効くので開発も捗ると思います。

```python
from honomara_members_site.model import Member
Member.query.order_by(Member.year.desc()).limit(10).all()
```

なお、SQLを直接書きたい場合は
以下のようにすればできます。

```bash
mysql -u root --password='honomara' -D honomara
```

また、mysqlが途中でフリーズした場合は

`sudo systemctl restart mysql`  
とすればなおると思います。

プルリクお待ちしてまーす